In [1]:
import polars as pl
import pandas as pd
import json
import os
import re

In [30]:
def read_jsonl_file_line_by_line(file_path):
    data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"JSON decode error in {file_path}: {e}")
        return data
    except Exception as e:
        print(f"Error opening {file_path}: {e}")
        return None

In [31]:
pattern = r'diff --git a/.+? b/(.+)'

In [32]:
def process_experiments(experiments, base_dir):
    all_preds = {}
    pattern = r'(?:diff --git a/.+? b/|--- a/)(.+)'

    for exp in experiments:
        file_path = os.path.join(base_dir, exp, 'all_preds.jsonl')
        results_file_path = os.path.join(base_dir, exp, 'results', 'results.json')
        
        try:
            with open(results_file_path, 'r', encoding='utf-8') as f:
                results_data = json.load(f)
                resolved_list = results_data.get('resolved', [])
        except Exception as e:
            print(f"Error reading results file for experiment {exp}: {e}")
            resolved_list = []

        df_data = read_jsonl_file_line_by_line(file_path)
        
        if df_data is not None:
            df = pl.DataFrame(df_data)

            df = df.with_columns(
                pl.lit(exp).alias('model_name_or_path')
            )

            if 'model_patch' in df.columns:
                df = df.with_columns(
                    pl.col('model_patch').str.extract(pattern, 1).alias('file_changed')
                )
            
            df = df.with_columns(
                pl.col('instance_id').is_in(resolved_list).alias('resolved')
            )
            
            df = df.select(['model_name_or_path', 'instance_id', 'file_changed', 'model_patch', 'resolved'])
            all_preds[exp] = df

    combined_df = pl.concat(list(all_preds.values()))
    return combined_df

In [33]:
def analyze_bench(bench_df, swe_df_small):
    merged_df = bench_df.join(swe_df_small, on='instance_id', how='inner')

    merged_df = merged_df.with_columns([
        pl.col('file_changed').str.strip_chars().str.replace_all(r'\\', '/').alias('file_changed_norm'),
        pl.col('true_patch_file').str.strip_chars().str.replace_all(r'\\', '/').alias('true_patch_file_norm')
    ])

    merged_df = merged_df.with_columns(
        (pl.col('file_changed_norm') == pl.col('true_patch_file_norm')).alias('is_same_file')
    )

    num_same_files = merged_df['is_same_file'].sum()
    total_instances = merged_df.shape[0]
    percentage = (num_same_files / total_instances) * 100 if total_instances > 0 else 0

    print(f"Overall Statistics:")
    print(f"Number of instances where 'file_changed' == 'true_patch_file': {num_same_files}")
    print(f"Total number of instances: {total_instances}")
    print(f"Percentage of matches: {percentage:.2f}%\n")

    for resolved_status in [True, False]:
        status_df = merged_df.filter(pl.col('resolved') == resolved_status)
        num_same_files = status_df['is_same_file'].sum()
        total_instances = status_df.shape[0]
        percentage = (num_same_files / total_instances) * 100 if total_instances > 0 else 0

        status = "Resolved" if resolved_status else "Not Resolved"
        print(f"{status} Instances:")
        print(f"Number of instances where 'file_changed' == 'true_patch_file': {num_same_files}")
        print(f"Total number of instances: {total_instances}")
        print(f"Percentage of matches: {percentage:.2f}%\n")

    model_stats = merged_df.group_by(['model_name_or_path', 'resolved']).agg([
        pl.len().alias('total_instances'),
        pl.col('is_same_file').sum().alias('num_same_files'),
        (pl.col('is_same_file').sum() / pl.len() * 100).alias('percentage_matches')
    ])

    print("Model Statistics by Resolved Status:")
    print(model_stats.filter(pl.col('resolved') == False).sort('percentage_matches'))
    return merged_df, model_stats


In [34]:
swe_df = pl.read_parquet('hf://datasets/princeton-nlp/SWE-bench/data/test-00000-of-00001.parquet')
swe_bench_lite_df = pl.read_parquet('hf://datasets/princeton-nlp/SWE-bench_Lite/data/test-00000-of-00001.parquet')
swe_bench_verified_df = pl.read_parquet('hf://datasets/princeton-nlp/SWE-bench_Verified/data/test-00000-of-00001.parquet')

# Lite bench

## Experiments (6 best)

- 20240702_codestory_aide_mixed
- 20240820_honeycomb
- 20240627_abanteai_mentatbot_gpt4o
- 20240811_gru
- 20240829_Isoform
- 20240806_SuperCoder2

In [35]:
lite_experiments = [
    '20240702_codestory_aide_mixed',
    '20240820_honeycomb',
    '20240627_abanteai_mentatbot_gpt4o',
    '20240811_gru',
    '20240829_Isoform',
    '20240806_SuperCoder2.0',
]

lite_dir = './experiments/evaluation/lite/'

lite_bench_df = process_experiments(lite_experiments, lite_dir)
lite_bench_df

model_name_or_path,instance_id,file_changed,model_patch,resolved
str,str,str,str,bool
"""20240702_codestory_aide_mixed""","""matplotlib__matplotlib-25433""","""lib/matplotlib/widgets.py""","""diff --git a/lib/matplotlib/widgets.py b/lib/matplotlib/widgets.py index ebe5da43e2..fd6f56f003 1006…",false
"""20240702_codestory_aide_mixed""","""django__django-16820""","""django/db/migrations/autodetector.py""","""diff --git a/django/db/migrations/autodetector.py b/django/db/migrations/autodetector.py index 23c97…",false
"""20240702_codestory_aide_mixed""","""django__django-15781""","""django/core/management/base.py""","""diff --git a/django/core/management/base.py b/django/core/management/base.py index d37d43d5c5..8a866…",false
"""20240702_codestory_aide_mixed""","""sympy__sympy-18087""","""sympy/simplify/trigsimp.py""","""diff --git a/sympy/simplify/trigsimp.py b/sympy/simplify/trigsimp.py index 7ca05f0304..b4ea4826a9 10…",false
"""20240702_codestory_aide_mixed""","""pytest-dev__pytest-5495""","""src/_pytest/_code/code.py""","""diff --git a/src/_pytest/_code/code.py b/src/_pytest/_code/code.py index b0b4d6531..4a2a39d8a 100644…",false
…,…,…,…,…
"""20240806_SuperCoder2.0""","""scikit-learn__scikit-learn-25570""","""sklearn/compose/_column_transformer.py""","""diff --git a/sklearn/compose/_column_transformer.py b/sklearn/compose/_column_transformer.py index a…",false
"""20240806_SuperCoder2.0""","""sphinx-doc__sphinx-8801""","""sphinx/ext/autodoc/__init__.py""","""diff --git a/sphinx/ext/autodoc/__init__.py b/sphinx/ext/autodoc/__init__.py index dc413b5c2..c944de…",false
"""20240806_SuperCoder2.0""","""sympy__sympy-14396""","""sympy/polys/polyoptions.py""","""diff --git a/sympy/polys/polyoptions.py b/sympy/polys/polyoptions.py index 6e504a3057..93eb1a451d 10…",false


In [36]:
swe_bench_lite_df = swe_bench_lite_df.with_columns(
    pl.col('patch').str.extract(pattern, 1).alias('true_patch_file')
)
swe_df_small = swe_bench_lite_df.select(['instance_id', 'true_patch_file'])

lite_merged_df, lite_model_stats = analyze_bench(lite_bench_df, swe_df_small)

Overall Statistics:
Number of instances where 'file_changed' == 'true_patch_file': 1196
Total number of instances: 1797
Percentage of matches: 66.56%

Resolved Instances:
Number of instances where 'file_changed' == 'true_patch_file': 612
Total number of instances: 672
Percentage of matches: 91.07%

Not Resolved Instances:
Number of instances where 'file_changed' == 'true_patch_file': 584
Total number of instances: 1125
Percentage of matches: 51.91%

Model Statistics by Resolved Status:
shape: (6, 5)
┌───────────────────────────────┬──────────┬─────────────────┬────────────────┬────────────────────┐
│ model_name_or_path            ┆ resolved ┆ total_instances ┆ num_same_files ┆ percentage_matches │
│ ---                           ┆ ---      ┆ ---             ┆ ---            ┆ ---                │
│ str                           ┆ bool     ┆ u32             ┆ u32            ┆ f64                │
╞═══════════════════════════════╪══════════╪═════════════════╪════════════════╪════════════

# Full bench

## Experiments (5 best)

- 20240820_honeycomb
- 20240509_amazon-q-developer-agent-20240430-dev
- 20240617_factory_code_droid
- 20240628_autocoderover-v20240620
- 20240620_sweagent_claude3.5sonnet

In [37]:
full_experiments = [
    '20240820_honeycomb',
    '20240509_amazon-q-developer-agent-20240430-dev',
    '20240617_factory_code_droid',
    '20240628_autocoderover-v20240620',
    '20240620_sweagent_claude3.5sonnet',
]

full_dir = './experiments/evaluation/test/'

full_bench_df = process_experiments(full_experiments, full_dir)

In [38]:
swe_df = swe_df.with_columns(
    pl.col('patch').str.extract(pattern, 1).alias('true_patch_file')
)
swe_df_small = swe_df.select(['instance_id', 'true_patch_file'])

full_merged_df, full_model_stats = analyze_bench(full_bench_df, swe_df_small)

Overall Statistics:
Number of instances where 'file_changed' == 'true_patch_file': 6490
Total number of instances: 11329
Percentage of matches: 57.29%

Resolved Instances:
Number of instances where 'file_changed' == 'true_patch_file': 1765
Total number of instances: 2113
Percentage of matches: 83.53%

Not Resolved Instances:
Number of instances where 'file_changed' == 'true_patch_file': 4725
Total number of instances: 9216
Percentage of matches: 51.27%

Model Statistics by Resolved Status:
shape: (5, 5)
┌───────────────────────────────┬──────────┬─────────────────┬────────────────┬────────────────────┐
│ model_name_or_path            ┆ resolved ┆ total_instances ┆ num_same_files ┆ percentage_matches │
│ ---                           ┆ ---      ┆ ---             ┆ ---            ┆ ---                │
│ str                           ┆ bool     ┆ u32             ┆ u32            ┆ f64                │
╞═══════════════════════════════╪══════════╪═════════════════╪════════════════╪════════

# Verified bench
## Experiments (All)

In [39]:
verified_dir = './experiments/evaluation/verified/'

verified_experiments = os.listdir(verified_dir)
verified_experiments

['20231010_rag_claude2',
 '20231010_rag_gpt35',
 '20231010_rag_swellama13b',
 '20231010_rag_swellama7b',
 '20240402_rag_claude3opus',
 '20240402_rag_gpt4',
 '20240402_sweagent_claude3opus',
 '20240402_sweagent_gpt4',
 '20240509_amazon-q-developer-agent-20240430-dev',
 '20240615_appmap-navie_gpt4o',
 '20240617_factory_code_droid',
 '20240620_sweagent_claude3.5sonnet',
 '20240628_autocoderover-v20240620',
 '20240721_amazon-q-developer-agent-20240719-dev',
 '20240728_sweagent_gpt4o',
 '20240820_epam-ai-run-gpt-4o',
 '20240820_honeycomb',
 '20240824_gru']

In [40]:
verified_df = process_experiments(verified_experiments, verified_dir)
verified_df

Error opening ./experiments/evaluation/verified/20240620_sweagent_claude3.5sonnet\all_preds.jsonl: [Errno 2] No such file or directory: './experiments/evaluation/verified/20240620_sweagent_claude3.5sonnet\\all_preds.jsonl'


model_name_or_path,instance_id,file_changed,model_patch,resolved
str,str,str,str,bool
"""20231010_rag_claude2""","""sympy__sympy-12419""","""sympy/matrices/expressions/matexpr.py""",""" Here is a patch file that fixes the issue of the sum of an identity matrix being 0 instead of n: `…",false
"""20231010_rag_claude2""","""sympy__sympy-18698""","""sympy/polys/polytools.py""","""--- a/sympy/polys/polytools.py +++ b/sympy/polys/polytools.py @@ -1537,7 +1537,7 @@ if self…",false
"""20231010_rag_claude2""","""sympy__sympy-15599""","""sympy/core/mod.py""","""--- a/sympy/core/mod.py +++ b/sympy/core/mod.py @@ -114,6 +114,11 @@ both_l[isinsta…",false
"""20231010_rag_claude2""","""astropy__astropy-14539""","""astropy/io/fits/diff.py""","""--- a/astropy/io/fits/diff.py +++ b/astropy/io/fits/diff.py @@ -727,6 +727,9 @@ if not …",false
"""20231010_rag_claude2""","""sympy__sympy-16450""","""sympy/assumptions/posify.py""","""--- a/sympy/assumptions/posify.py +++ b/sympy/assumptions/posify.py @@ -14,6 +14,7 @@ def posify(x):…",false
…,…,…,…,…
"""20240824_gru""","""sympy__sympy-21612""","""sympy/parsing/latex/_parse_latex_antlr.py""","""diff --git a/sympy/parsing/latex/_parse_latex_antlr.py b/sympy/parsing/latex/_parse_latex_antlr.py i…",false
"""20240824_gru""","""sympy__sympy-22080""","""sympy/plotting/experimental_lambdify.py""","""diff --git a/sympy/plotting/experimental_lambdify.py b/sympy/plotting/experimental_lambdify.py index…",false
"""20240824_gru""","""sympy__sympy-23413""","""sympy/polys/matrices/normalforms.py""","""diff --git a/sympy/polys/matrices/normalforms.py b/sympy/polys/matrices/normalforms.py index 30f5534…",false


In [41]:
swe_bench_verified_df = swe_bench_verified_df.with_columns(
    pl.col('patch').str.extract(pattern, 1).alias('true_patch_file')
)
swe_df_verified = swe_bench_verified_df.select(['instance_id', 'true_patch_file'])

verified_merged_df, verified_model_stats = analyze_bench(verified_df, swe_df_verified)

Overall Statistics:
Number of instances where 'file_changed' == 'true_patch_file': 4513
Total number of instances: 8512
Percentage of matches: 53.02%

Resolved Instances:
Number of instances where 'file_changed' == 'true_patch_file': 1625
Total number of instances: 1784
Percentage of matches: 91.09%

Not Resolved Instances:
Number of instances where 'file_changed' == 'true_patch_file': 2888
Total number of instances: 6728
Percentage of matches: 42.93%

Model Statistics by Resolved Status:
shape: (17, 5)
┌───────────────────────────────┬──────────┬─────────────────┬────────────────┬────────────────────┐
│ model_name_or_path            ┆ resolved ┆ total_instances ┆ num_same_files ┆ percentage_matches │
│ ---                           ┆ ---      ┆ ---             ┆ ---            ┆ ---                │
│ str                           ┆ bool     ┆ u32             ┆ u32            ┆ f64                │
╞═══════════════════════════════╪══════════╪═════════════════╪════════════════╪════════

In [44]:
pl.Config.set_tbl_rows(100)

polars.config.Config

In [49]:
verified_model_stats.filter(pl.col('resolved') == False).sort('percentage_matches')

model_name_or_path,resolved,total_instances,num_same_files,percentage_matches
str,bool,u32,u32,f64
"""20231010_rag_swellama7b""",false,489,70,14.314928
"""20231010_rag_swellama13b""",false,475,68,14.315789
"""20240402_sweagent_claude3opus""",false,493,145,29.411765
"""20231010_rag_gpt35""",false,498,163,32.730924
"""20240402_sweagent_gpt4""",false,390,132,33.846154
"""20240728_sweagent_gpt4o""",false,349,131,37.535817
"""20240615_appmap-navie_gpt4o""",false,368,173,47.01087
"""20240402_rag_gpt4""",false,486,229,47.119342
"""20231010_rag_claude2""",false,478,234,48.953975
